In [2]:
%cd /hdd/yuchen/StableDiffusionReconstruction/codes/utils

/hdd/yuchen/StableDiffusionReconstruction/codes/utils


In [5]:
import argparse
import os
import numpy as np
import pandas as pd
from nsd_access import NSDAccess
import scipy.io
import argparse, os
import PIL
import torch
import numpy as np
from omegaconf import OmegaConf
from tqdm import tqdm
from einops import repeat
from torch import autocast
from contextlib import nullcontext
# from pytorch_lightning import seed_everything
from nsd_access import NSDAccess
from PIL import Image
from ldm.util import instantiate_from_config
from ldm.models.diffusion.ddim import DDIMSampler
import scipy.io
from einops import rearrange
import random
from torchmetrics.image.lpip import LearnedPerceptualImagePatchSimilarity
from skimage.metrics import structural_similarity


In [4]:
seed = 42
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
torch.cuda.empty_cache()

In [33]:
def compare_one_set(i, lpips, folder_path):
    with torch.no_grad():
        base = str(f'{i:05}')
        orig = folder_path + f'{base}_org.png'
        orig_ = Image.open(orig).resize((512,512), Image.Resampling.BILINEAR)
        img2 = np.array(orig_)
        
        psm, ssim, pcc = 0, 0, 0
        for j in range(4):
            img = folder_path + f'{base}_{j:03}.png'
            img_ = Image.open(img)
            img1 = np.array(img_)
            
            if img1.shape[-1] == 3: img1 = rearrange(img1, 'w h c -> c w h')
            if img2.shape[-1] == 3: img2 = rearrange(img2, 'w h c -> c w h')
            if img1.max() > 1: img1 = img1 / 255.0
            if img2.max() > 1: img2 = img2 / 255.0
            if len(img1.shape) == 3: img1 = np.expand_dims(img1, axis=0)
            if len(img2.shape) == 3: img2 = np.expand_dims(img2, axis=0)
                
            psm += lpips(torch.FloatTensor(img1).to(device), torch.FloatTensor(img2).to(device)).item()
            ssim += structural_similarity(img1.squeeze(),img2.squeeze(), data_range=1, channel_axis=0)
            pcc += np.corrcoef(img1.reshape(-1), img2.reshape(-1))[0, 1]
            
    return psm/4, ssim/4, pcc/4

folder_path = '/hdd/yuchen/decoded/image-cvpr/subj01/samples/'
device = "cuda:1" if torch.cuda.is_available() else "cpu"
lpips = LearnedPerceptualImagePatchSimilarity(net_type='alex', normalize=True).to(device)

score = {'psm': [], 'ssim': [], 'pcc': []}
for i in range(30):
    test_score = compare_one_set(i, lpips, folder_path)
    score['psm'].append(test_score[0])
    score['ssim'].append(test_score[1])
    score['pcc'].append(test_score[2])

In [39]:
np.mean(score['psm']), np.mean(score['ssim']), np.mean(score['pcc'])

(0.6775105200707913, 0.2232460986445165, 0.27371258050492725)

In [26]:
import numpy as np  
from skimage.metrics import structural_similarity

img1 = np.random.randint(0, 255, size=(200, 200, 3)).astype(np.float32)
img2 = np.random.randint(0, 255, size=(200, 200, 3)).astype(np.float32)

ssim_score = structural_similarity(img1, img2, multichannel=True) #score: 0.0018769083894301646


ValueError: win_size exceeds image extent. Either ensure that your images are at least 7x7; or pass win_size explicitly in the function call, with an odd value less than or equal to the smaller side of your images. If your images are multichannel (with color channels), set channel_axis to the axis number corresponding to the channels.

In [10]:
score

NameError: name 'score' is not defined

In [55]:
img1 = np.array(orig_)
img2 = np.array(img_)

In [80]:
img1 = np.array(img_)
img2 = np.array(orig_.resize((img1.shape[0], img1.shape[1])))

In [85]:
img1 = np.array(img_)
img2 = np.array(orig_.resize((img1.shape[0], img1.shape[1])))

device = "cuda:0" if torch.cuda.is_available() else "cpu"
lpips = LearnedPerceptualImagePatchSimilarity(net_type='alex', normalize=True).to(device)

# @torch.no_grad()
with torch.no_grad():
    if img1.shape[-1] == 3:
        img1 = rearrange(img1, 'w h c -> c w h')
        img2 = rearrange(img2, 'w h c -> c w h')
    img1 = img1 / 255.0
    img2 = img2 / 255.0
    img1 = np.expand_dims(img1, axis=0)
    img2 = np.expand_dims(img2, axis=0)
    t = lpips(torch.FloatTensor(img1).to(device), torch.FloatTensor(img2).to(device)).item()

In [84]:
t

0.0

In [22]:
LearnedPerceptualImagePatchSimilarity(net_type='alex').to(device)

LearnedPerceptualImagePatchSimilarity(
  (net): NoTrainLpips(
    (scaling_layer): ScalingLayer()
    (net): alexnet(
      (slice1): Sequential(
        (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
        (1): ReLU(inplace=True)
      )
      (slice2): Sequential(
        (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
        (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
        (4): ReLU(inplace=True)
      )
      (slice3): Sequential(
        (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
        (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (7): ReLU(inplace=True)
      )
      (slice4): Sequential(
        (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (9): ReLU(inplace=True)
      )
      (slice5): Sequential(
        (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  

In [5]:
test = scipy.io.loadmat( '/hdd/yuchen/nsd/nsddata/experiments/nsd/nsd_expdesign.mat')

In [6]:
test.keys()

dict_keys(['__header__', '__version__', '__globals__', 'basiccnt', 'masterordering', 'sharedix', 'stimpattern', 'subjectim'])

In [13]:
test['subjectim']  #  1-base index!

array([[ 2951,  2991,  3050, ..., 19472, 49508, 11124],
       [ 2951,  2991,  3050, ..., 29015, 40443,  7566],
       [ 2951,  2991,  3050, ..., 40627, 60549, 60491],
       ...,
       [ 2951,  2991,  3050, ..., 22828, 64982, 39213],
       [ 2951,  2991,  3050, ..., 68368, 68973, 16416],
       [ 2951,  2991,  3050, ..., 24254, 14948, 42093]], dtype=int32)

In [31]:
len(np.unique(test['subjectim'][0]))

10000

In [24]:
t = np.array([1,1,2,3])


array([1, 2, 3])

In [16]:
test['sharedix']

array([[ 2951,  2991,  3050,  3078,  3147,  3158,  3165,  3172,  3182,
         3387,  3435,  3450,  3490,  3627,  3683,  3688,  3730,  3810,
         3843,  3848,  3857,  3914,  3952,  4052,  4059,  4130,  4157,
         4250,  4326,  4424,  4437,  4613,  4668,  4691,  4769,  4787,
         4836,  4870,  4893,  4931,  5035,  5107,  5205,  5286,  5302,
         5339,  5428,  5460,  5503,  5543,  5575,  5584,  5603,  5715,
         5879,  5891,  6133,  6200,  6223,  6432,  6445,  6490,  6515,
         6522,  6525,  6559,  6641,  6714,  6802,  7008,  7040,  7121,
         7208,  7337,  7367,  7410,  7419,  7481,  7655,  7660,  7841,
         7860,  7945,  7949,  7955,  8007,  8110,  8205,  8226,  8263,
         8275,  8319,  8388,  8395,  8416,  8436,  8466,  8510,  8632,
         8647,  8808,  8844,  8926,  8934,  9028,  9049,  9148,  9231,
         9435,  9463,  9681,  9723,  9805,  9848,  9866,  9918,  9979,
        10007, 10047, 10065, 10106, 10108, 10245, 10394, 10472, 10508,
      